# Interactive Learn.co Backup and Nuke

# <span style="color:blue">Step 1</span>

#### Before you begin, use the following instructions to create a personal access token on GitHub:
https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line

#### Use the following settings for the Authorization Scope of the token:

![](github-token-a.png)
![](github-token-b.png)

# <span style="color:blue">Step 2</span>

#### In the same directory as this Notebook, create a Python file, secrets.py, and in there create variables with the following values:

    user = 'your_github_user_name'
    token = 'your_access_token'

#### Where "your_github_user_name" is replaced by your GitHub user name, and "your_access_token" is replaced by the personal access token you just generated.

In [2]:
import requests
import secrets
import re
import time
import json
import os

In [3]:
# Get first page of results for repo list
headers = {'Authorization' : f'token {secrets.token}'}
r = requests.get(f'https://api.github.com/user/repos', headers=headers)

# Read response header to find total number of pages of results
links = r.headers['Link'].split(',')
last_page_link = [link for link in links if "last" in link][0]
last_page_number = int(re.search(r'\?page=(\d+)', last_page_link).group(1))

In [4]:
results = []

for i in range(1, last_page_number+1):
    headers = {'Authorization' : f'token {secrets.token}'}
    params = {'page' : i}
    r = requests.get(f'https://api.github.com/user/repos', headers=headers, params=params)
    time.sleep(1) # pause to avoid lockout
    results += json.loads(r.content)

# <span style="color:blue">Step 3</span>

#### Modify the following two lists below to match your Learn.co repositories. Most of them have a six-digit date code (the start-date of your cohort) which can be used to identify them.

In [5]:
# We will grab all repos matching these substrings
learnco_indicator_substrings = ['000', '082619', '100719']

# We will ignore all repos matching these substrings
exclusion_substrings = []

In [10]:
# Now let's filter for the Learn.co repos
filter_func = lambda x: (x['full_name'].startswith(secrets.user) and not
                        (len([s for s in exclusion_substrings if s in x['full_name']]) > 0) and
                        (len([s for s in learnco_indicator_substrings if s in x['full_name']]) > 0))
learnco_repos = list(filter(filter_func, results))

# And print them out
for repo in learnco_repos:
    print(repo['full_name'])

jmsmdy/applying-gradient-descent-data-science-intro-000
jmsmdy/applying-gradient-descent-lab-data-science-intro-000
jmsmdy/applying-nearest-neighbors-data-science-intro-000
jmsmdy/calculating-distance-lab-data-science-intro-000
jmsmdy/calculating-lines-data-science-intro-000
jmsmdy/calculus-derivation-gradient-descent-data-science-intro-000
jmsmdy/conditionals-python-lab-data-science-intro-000
jmsmdy/conditionals-python-readme-data-science-intro-000
jmsmdy/derivative-chain-rule-data-science-intro-000
jmsmdy/derivative-rules-ds-apply-000
jmsmdy/ds-k-means-clustering-lab-nyc-ds-100719
jmsmdy/ds-word-vectorization-lab-nyc-ds-100719
jmsmdy/dsc-accessing-data-with-pandas-lab-nyc-ds-082619
jmsmdy/dsc-accessing-data-with-pandas-nyc-ds-082619
jmsmdy/dsc-analyzing-macbeth-project-nyc-ds-082619
jmsmdy/dsc-bash-shell-nyc-ds-082619
jmsmdy/dsc-bernoulli-and-binomial-distribution-lab-nyc-ds-082619
jmsmdy/dsc-bernoulli-and-binomial-distribution-nyc-ds-082619
jmsmdy/dsc-bias-variance-trade-off-lab-nyc

# <span style="color:red">WARNING: BE CAREFUL IN THIS STEP</span>

# <span style="color:blue">Step 4</span> 
#### Before continuing, manually review the list of repos above, and verify that all of them are Learn.co repos. If not, Repeat Step 3 until only Learn.co repos are left in this list. <span style="color:red">Every repo in the list above will be deleted from GitHub.</span> 

In [11]:
def backup(repo_full_name):
    working_dir = os.getcwd()
    repo_name = repo_full_name.split('/')[-1]
    repo_path = os.path.join(working_dir, 'BACKUP', repo_name)
    upstream_url = f'https://github.com/{secrets.user}/{repo_name}.git'
    
    os.makedirs(repo_path)
    exit_code_a = os.system(f'git clone --mirror {upstream_url} {os.path.join(repo_path, ".git")}')
    os.chdir(repo_path)
    exit_code_b = os.system('git config --bool core.bare false')
    exit_code_c = os.system('git checkout master')
    os.chdir(working_dir)
    
    if exit_code_a or exit_code_b or exit_code_c:
        print(f'Backing up {repo_full_name} failed!')
        return False
    else:
        return True
              
def delete_repo(repo_full_name):
    repo_name = repo_full_name.split('/')[-1]
    repo_url = f'https://api.github.com/repos/{secrets.user}/{repo_name}'
    headers = {'Authorization' : f'token {secrets.token}'}
    return requests.delete(repo_url, headers=headers)
                            
def backup_and_delete_repo(repo_full_name):
    successful = backup(repo_full_name)
    if successful:
        delete_repo(repo_full_name)
        print(f'Repo {repo_full_name} backed up and deleted.')
    else:
        print(f'Repo {repo_full_name} not deleted.')

# <span style="color:blue">Step 5</span>

#### Run the below command to backup and delete all repos in the list above. Keep an eye out for errors (you may need to manually intervene).

In [12]:
for repo in learnco_repos:
    backup_and_delete_repo(repo['full_name'])

Repo jmsmdy/applying-gradient-descent-data-science-intro-000 backed up and deleted.
Repo jmsmdy/applying-gradient-descent-lab-data-science-intro-000 backed up and deleted.
Repo jmsmdy/applying-nearest-neighbors-data-science-intro-000 backed up and deleted.
Repo jmsmdy/calculating-distance-lab-data-science-intro-000 backed up and deleted.
Repo jmsmdy/calculating-lines-data-science-intro-000 backed up and deleted.
Repo jmsmdy/calculus-derivation-gradient-descent-data-science-intro-000 backed up and deleted.
Repo jmsmdy/conditionals-python-lab-data-science-intro-000 backed up and deleted.
Repo jmsmdy/conditionals-python-readme-data-science-intro-000 backed up and deleted.
Repo jmsmdy/derivative-chain-rule-data-science-intro-000 backed up and deleted.
Repo jmsmdy/derivative-rules-ds-apply-000 backed up and deleted.
Repo jmsmdy/ds-k-means-clustering-lab-nyc-ds-100719 backed up and deleted.
Repo jmsmdy/ds-word-vectorization-lab-nyc-ds-100719 backed up and deleted.
Repo jmsmdy/dsc-accessing-d

Repo jmsmdy/dsc-k-nearest-neighbors-lab-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-k-nearest-neighbors-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-knn-intro-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-knn-with-scikit-learn-lab-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-knn-with-scikit-learn-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-lambda-functions-lab-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-lambda-functions-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-law-of-total-probability-lab-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-law-of-total-probability-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-learn-lessons-lab-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-learn-lessons-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-linear-regression-section-intro-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-linear-regression-section-recap-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-linea

Repo jmsmdy/dsc-variable-assignment-intro-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-variable-assignment-lab-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-visualizations-section-intro-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-visualizations-section-recap-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-while-loops-break-and-continue-lab-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-while-loops-break-and-continue-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-working-with-known-json-schemas-lab-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-working-with-known-json-schemas-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/dsc-z-score-p-value-nyc-ds-082619 backed up and deleted.
Repo jmsmdy/evaluating-regression-lines-data-science-intro-000 backed up and deleted.
Repo jmsmdy/evaluating-regression-lines-lab-data-science-intro-000 backed up and deleted.
Repo jmsmdy/excel-to-python-data-science-intro-000 backed up and deleted.
Repo jmsmdy/exc